In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

---

## Split the Data into Training and Testing Sets

In [2]:
# Load the test data into a Pandas DataFrame
df_test = pd.read_csv("https://machine-failure-data-20230822-craiguo.s3.us-west-2.amazonaws.com/test.csv")
df_test.drop(columns = ["id"], inplace = True)
# Display sample data
df_test.head(10)

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,136429,L50896,L,302.3,311.5,1499,38.0,60,0,0,0,0,0
1,136430,L53866,L,301.7,311.0,1713,28.8,17,0,0,0,0,0
2,136431,L50498,L,301.3,310.4,1525,37.7,96,0,0,0,0,0
3,136432,M21232,M,300.1,309.6,1479,47.6,5,0,0,0,0,0
4,136433,M19751,M,303.4,312.3,1515,41.3,114,0,0,0,0,0
5,136434,L49438,L,299.1,308.3,1489,38.2,139,0,0,0,0,0
6,136435,L50286,L,299.8,309.1,1429,39.9,207,0,0,0,0,0
7,136436,L52589,L,302.7,312.4,1540,46.2,17,0,0,0,0,0
8,136437,H37110,H,300.7,311.9,1613,36.0,12,0,0,0,0,0
9,136438,L54886,L,300.5,311.4,1708,32.2,57,0,0,0,0,0


In [3]:
# Load the test data into a Pandas DataFrame
df_train = pd.read_csv("https://machine-failure-data-20230822-craiguo.s3.us-west-2.amazonaws.com/train.csv")
df_train.drop(columns = ["id"], inplace = True)
# Display sample data
df_train.head(10)

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0
5,5,M24300,M,298.4,308.9,1429,42.1,65,0,0,0,0,0,0
6,6,L56736,L,299.6,311.0,1413,42.9,156,0,0,0,0,0,0
7,7,L55488,L,298.7,310.1,1609,38.1,67,0,0,0,0,0,0
8,8,L56397,L,297.7,308.8,1578,35.2,13,0,0,0,0,0,0
9,9,L55067,L,300.5,312.3,1447,53.3,98,0,0,0,0,0,0


In [4]:
# Look at nunique counts for train
df_train.nunique()

id                         136429
Product ID                   9976
Type                            3
Air temperature [K]            95
Process temperature [K]        81
Rotational speed [rpm]        952
Torque [Nm]                   611
Tool wear [min]               246
Machine failure                 2
TWF                             2
HDF                             2
PWF                             2
OSF                             2
RNF                             2
dtype: int64

In [5]:
# Look at nunique counts for test
df_test.nunique()

id                         90954
Product ID                  9909
Type                           3
Air temperature [K]           92
Process temperature [K]       84
Rotational speed [rpm]       946
Torque [Nm]                  595
Tool wear [min]              246
TWF                            2
HDF                            2
PWF                            2
OSF                            2
RNF                            2
dtype: int64

In [6]:
# concatenate the dataframes to use pd.get_dummies()

combined_df = pd.concat([df_train, df_test], sort=False)
combined_df.tail()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
90949,227378,L51130,L,302.3,311.4,1484,40.4,15,NaN,0,0,0,0,0
90950,227379,L47783,L,297.9,309.8,1542,33.8,31,NaN,0,0,0,0,0
90951,227380,L48097,L,295.6,306.2,1501,41.4,187,NaN,0,0,0,0,0
90952,227381,L48969,L,298.1,307.8,1534,40.3,69,NaN,0,0,0,0,0
90953,227382,L52525,L,303.5,312.8,1534,36.1,92,NaN,0,0,0,0,0


In [7]:
#use pd.get_dummies() for 'Product ID' and 'Type' and split test and train sets again

dummies_df = pd.get_dummies(combined_df)

col = df_train.shape[0]

dummy_train = dummies_df.iloc[0:col]
dummy_test = dummies_df.iloc[col:]

### Step 2: Create features and target arrays

In [8]:
# Split our preprocessed data into our features and target arrays, 
# using only training data to be able to evaluate performace of the model directly

y = dummy_train['Machine failure']
X = dummy_train.drop(columns = 'Machine failure')

In [9]:
# Review the y variable Series
y[:5]

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Machine failure, dtype: float64

In [10]:
# Review the X variable DataFrame
X.head()

,id,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,...,Product ID_M24846,Product ID_M24847,Product ID_M24849,Product ID_M24851,Product ID_M24855,Product ID_M24857,Product ID_M24859,Type_H,Type_L,Type_M
0,0,300.6,309.6,1596,36.1,140,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,302.6,312.1,1759,29.1,200,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,299.3,308.5,1805,26.5,25,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3,301.0,310.9,1524,44.3,197,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,298.0,309.0,1641,35.4,34,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Step 3: Split the data into training and testing datasets by using `train_test_split`.

In [11]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
#Scale the data using StandardScaler()
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Create a Random Forest Model

In [12]:
 # Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [45]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

### Step 3: Evaluate the model’s performance by doing the following:

* Generate a confusion matrix.

* Calculate the accuracy score.

* Print the classification report.

In [46]:
 # Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

[[18663   102]
 [   56   563]]


In [47]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384

